In [1]:
# Project consists of classifying a test set of dogs and cats after training it from a training set
# Made by:
# Sheng Hao Liu, 260585377
# Alex Makrigiorgos, 260573080

# All imports
import numpy as np
import cv2
import os
import math
import matplotlib.pyplot as plt
import csv
# for sorting
from operator import itemgetter, attrgetter, methodcaller
# for getting all images in folder
import glob
# for svm classification
from sklearn import svm
from sklearn.externals import joblib
from skimage.feature import hog
# better padding
import pandas as pd

In [2]:

# PART1 Define all functions needed ====================================>

def get_img_folder(path):
    # path looks like 'C:/Users/Sheng/Jupyter Notebook/celeb/*.JPG'
    # get all images in folder: change directory as needed 
    cv_gre = []
    cv_col = []
    for img in glob.glob(path):
        gr = cv2.imread(img, 0)
#         gr = cv2.resize(gr, (128,128))
        cv_gre.append(gr)
        co = cv2.imread(img, 1)
#         co = cv2.resize(co, (128,128))
        cv_col.append(co)
    # return grey list then color list of images in a directory
    return cv_gre, cv_col;

def get_col_folder(path):
    # color only
    cv_col = []
    for img in glob.glob(path):
        co = cv2.imread(img, 1)
        cv_col.append(co)
    # return color only
    return cv_col;

def siftDesc(img):
    # extract features from an image using SIFT
    # Create a SIFT object
    sift = cv2.xfeatures2d.SIFT_create()
    # Detect SIFT features, with no masks
    keypoints = sift.detect(img,None)
    # limit features to 100 best
    # if there are less than 100, then all X < 100 are taken
    kp = sorted(keypoints, key=attrgetter('response'), reverse=True)
    kp = kp[:100]
    # returns 100 best keypoints & descriptors
    # if there are less, then returns 0s I believe
    keypoints, descriptors = sift.compute(img, kp)
    return descriptors;

def surfDesc(img):
    # extract features from an image using SURF
    surf = cv2.xfeatures2d.SURF_create()
    # Detect SIFT features, with no masks
    keypoints = surf.detect(img,None)
    kp = sorted(keypoints, key=attrgetter('response'), reverse=True)
    kp = kp[:100]
    # returns 50 best keypoints & descriptors
    keypoints, descriptors = surf.compute(img, kp)
    return descriptors;

def hogDesc(img):
    # extract features from an image using HOG
    hogging = cv2.HOGDescriptor((64,64), (16,16), (8,8), (8,8), 9)
    # these parameters will give me a shape of (1764,1)
    height = img.shape[0]
    width = img.shape[1]
    location = (int(height/2), int(width/2))
    # computing in a set location 
    # I want to believe that all cats/dogs are centered in image
    hdesc = hogging.compute(trainset2[do], (8,8), (8,8), (location,))
    return hdesc;

def siftDesc50(img):
    # extract features from an image using SIFT
    # Create a SIFT object
    sift = cv2.xfeatures2d.SIFT_create()
    # Detect SIFT features, with no masks
    keypoints = sift.detect(img,None)
    kp = sorted(keypoints, key=attrgetter('response'), reverse=True)
    kp = kp[:50]
    # returns 50 best keypoints & descriptors
    keypoints, descriptors = sift.compute(img, kp)
    return descriptors;

def surfDesc50(img):
    # extract features from an image using SURF
    surf = cv2.xfeatures2d.SURF_create()
    # Detect SIFT features, with no masks
    keypoints = surf.detect(img,None)
    kp = sorted(keypoints, key=attrgetter('response'), reverse=True)
    kp = kp[:50]
    # returns 50 best keypoints & descriptors
    keypoints, descriptors = surf.compute(img, kp)
    return descriptors;

def siftDesc25(img):
    # extract features from an image using SIFT
    # Create a SIFT object
    sift = cv2.xfeatures2d.SIFT_create()
    # Detect SIFT features, with no masks
    keypoints = sift.detect(img,None)
    kp = sorted(keypoints, key=attrgetter('response'), reverse=True)
    kp = kp[:25]
    # returns 25 best keypoints & descriptors
    keypoints, descriptors = sift.compute(img, kp)
    return descriptors;


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images


# <=================================================================== END OF PART1 



In [116]:
# Load training set
trainset2 = load_images_from_folder('C:/Users/Sheng/Jupyter Notebook/X_Train')

print("loading training set done!")
# Main below loads

loading training set done!


In [145]:
# Load testing set
testset2 = load_images_from_folder('C:/Users/Sheng/Jupyter Notebook/X_Test')

print("loading testing set done!")
# Main below loads

loading testing set done!


In [146]:
# To save as pkl for faster usage later
# grab discriptors for training
traindesc = []
# put all descriptors from the the training set categories into a huge descriptor array
for allimg in range(0, len(trainset2)):
    tempdesc = hogDesc(trainset2[allimg])
    traindesc.append(tempdesc)
    
# set back all as array
traindesc = np.asarray(traindesc)
print(traindesc.shape)

joblib.dump(traindesc, "trainingdeschog.pkl", compress=3)

print("training descriptors done!")

(5907, 1764, 1)
training descriptors done!


In [147]:
# To save as pkl for faster usage later
# grab discriptors for testing
testdesc = []
# put all descriptors from the the training set categories into a huge descriptor array
for allimg in range(0, len(testset2)):
    tempdesc = hogDesc(testset2[allimg])
    testdesc.append(tempdesc)
    
# set back all as array
testdesc = np.asarray(testdesc)
print(testdesc.shape)

joblib.dump(testdesc, "testingdeschog.pkl", compress=3)

print("testing descriptors done!")

(1477, 1764, 1)
testing descriptors done!


In [3]:
# Use this when dealing with SIFT/SURF
# load descriptors
traindesc = joblib.load("trainingdescsift100.pkl")
testdesc = joblib.load("testingdescsift100.pkl")

# load correct training ys
labels = []
with open('Y_Train.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        labels.append(row['Label'])

# Use Below when dealing with SIFT/SURF =========================================
# set third length
# SIFT = X, desc, 128
siftlen = 128
# SURF = X, desc, 64
surflen = 64
# how many features?
feat = 100
# set extension array as zeroes
newtrain = np.zeros((5907,feat,siftlen), dtype=np.float32)
newtest = np.zeros((1477,feat,siftlen), dtype=np.float32)

# copy training to extension
# extend1 = pd.DataFrame(extend1)
# extend2 = pd.DataFrame(extend2)
# traindesc = pd.DataFrame(traindesc)
# testdesc = pd.DataFrame(testdesc)

# extend1 = extend1.add(traindesc, fill_value = 0.0)
# extend2 = extend2.add(testdesc, fill_value = 0.0)
    
# extend1[:5907, :50, :64] = traindesc
# extend2[:1477, :50, :64] = testdesc
# print(extend1)
# print(extend2)
# Above doesn't work so just make a triple loop and add everything to the extended

# use this when dealing with SIFT or SURF =========================================
print("reshape in progress")
for one in range(0,5907):
    for two in range(0,traindesc[one].shape[0]):
        for three in range(0,siftlen):
            newtrain[one,two,three] = traindesc[one][two][three]
for one in range(0,1477):
    for two in range(0,testdesc[one].shape[0]):
        for three in range(0,siftlen):
            newtest[one,two,three] = testdesc[one][two][three]
# =================================================================================     

# set for SVMs
X = newtrain
print(X.shape)
xsamples, xx, xy = X.shape
# X2d = X
X2d = X.reshape((xsamples,xx*xy))
print(X2d.shape)
y = np.asarray(labels)
print(y.shape)

clf = svm.SVC(C=1.0, cache_size=1200, kernel='poly', degree=4, max_iter=-1, tol=0.0001)
clf.fit(X2d, y)

print("SVM Fitted!")

T = newtest
tsamples , tx, ty = T.shape
T2d = T.reshape((tsamples,tx*ty))

pred = clf.predict(T2d)
print(pred)
print(len(pred))

reshape in progress
(5907, 100, 128)
(5907, 12800)
(5907,)
SVM Fitted!
['1' '0' '1' ..., '1' '1' '0']
1477


In [152]:
# use this when dealing with HoG
# load descriptors
traindesc = joblib.load("trainingdeschog.pkl")
testdesc = joblib.load("testingdeschog.pkl")

# load correct training ys
labels = []
with open('Y_Train.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        labels.append(row['Label'])
        
# set for SVMs
X = traindesc
print(X.shape)
xsamples, xx, xy = X.shape
# X2d = X
X2d = X.reshape((xsamples,xx*xy))
print(X2d.shape)
y = np.asarray(labels)
print(y.shape)

clf = svm.SVC(C=1.0, cache_size=800, kernel='poly', degree=5, max_iter=-1, tol=0.0001)
clf.fit(X2d, y)

print("SVM Fitted!")

T = testdesc
tsamples , tx, ty = T.shape
T2d = T.reshape((tsamples,tx*ty))

pred = clf.predict(T2d)
print(pred)
print(len(pred))

(5907, 1764, 1)
(5907, 1764)
(5907,)
SVM Fitted!
['1' '1' '1' ..., '1' '1' '1']
1477


In [4]:
# Save in results

filename = []
with open('Y_Train.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        filename.append(row['Image'])
    fn100 = np.asarray(filename[:1477])

with open('TestResults.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['Image', 'Label'])
    for write in range(0, len(pred)):
        spamwriter.writerow([fn100[write], pred[write]])
        
print("Saving done!")

Saving done!


In [8]:
# better display for images/to see what has been saved in trainset2/testset2

# I have no idea why plt.imshow gives everything in blue
# for show in range(0,5):
#     plt.imshow(trainset1[show])
#     plt.show()
#     plt.imshow(trainset2[show])
#     plt.show()


# for show in range(0,5):
#     cv2.imshow('Frame ' + str(show), trainset2[show])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

NameError: name 'trainset2' is not defined

In [12]:
# see if file can be opened

with open('TestResults.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))


Image,Label
0.jpg,0
1.jpg,0
2.jpg,1
3.jpg,1
4.jpg,1
5.jpg,0
6.jpg,0
7.jpg,0
8.jpg,1
9.jpg,0
10.jpg,0
11.jpg,1
12.jpg,1
13.jpg,1
14.jpg,1
15.jpg,0
16.jpg,1
17.jpg,1
18.jpg,1
19.jpg,1
20.jpg,1
21.jpg,0
22.jpg,0
23.jpg,1
24.jpg,1
25.jpg,0
26.jpg,1
27.jpg,1
28.jpg,1
29.jpg,0
30.jpg,1
31.jpg,0
32.jpg,0
33.jpg,0
34.jpg,1
35.jpg,1
36.jpg,1
37.jpg,1
38.jpg,1
39.jpg,1
40.jpg,1
41.jpg,0
42.jpg,0
43.jpg,0
44.jpg,1
45.jpg,1
46.jpg,1
47.jpg,1
48.jpg,1
49.jpg,1
50.jpg,1
51.jpg,0
52.jpg,0
53.jpg,1
54.jpg,1
55.jpg,1
56.jpg,1
57.jpg,1
58.jpg,1
59.jpg,1
60.jpg,1
61.jpg,0
62.jpg,1
63.jpg,1
64.jpg,0
65.jpg,1
66.jpg,1
67.jpg,1
68.jpg,1
69.jpg,1
70.jpg,1
71.jpg,0
72.jpg,0
73.jpg,0
74.jpg,0
75.jpg,1
76.jpg,1
77.jpg,1
78.jpg,1
79.jpg,1
80.jpg,0
81.jpg,1
82.jpg,0
83.jpg,0
84.jpg,0
85.jpg,1
86.jpg,1
87.jpg,1
88.jpg,1
89.jpg,1
90.jpg,0
91.jpg,1
92.jpg,0
93.jpg,1
94.jpg,1
95.jpg,1
96.jpg,0
97.jpg,0
98.jpg,1
99.jpg,0
100.jpg,1
101.jpg,0
102.jpg,0
103.jpg,1
104.jpg,1
105.jpg,1
106.jpg,1
107.jpg,1
108.jpg,1
109.jpg,

In [91]:
# testing area

# how to save/load
# save as pkl
# joblib.dump((voc,clf), "imagereco.pkl", compress=3)
# reload vars
# voc, clf = joblib.load("imagereco.pkl")

# grab stuff with csv
# with open('Y_Train.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
#     for row in spamreader:
#         print(', '.join(row))
        
# with open('Y_Test_example.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
#     for row in spamreader:
#         print(', '.join(row))

# labels = []
# with open('Y_Train.csv') as csvfile:
#     reader = csv.DictReader(csvfile)
#     for row in reader:
#         labels.append(row['Label'])
#     label500 = labels[:500]


# test shapes
for total in range(0,5907):
    x = traindesc[total].shape[1]
    if (x != 64):
        print(total)
for total in range(0,1477):
    x = testdesc[total].shape[1]
    if (x != 64):
        print(total)

print(traindesc[1])
print(traindesc[1][2][3])
print(traindesc[1].shape)
print("finish")

# print(traindesc[2788])
# print(traindesc[2788].shape)
# print(traindesc[2788].shape[0])
# x = np.zeros((100-78,surflen), dtype=np.float32)
# print(x.shape)
# traindesc[2788] = np.vstack((traindesc[2788], x))
# print(traindesc[2788].shape[0])
# print("fixed")

# for total in range(0,5907):
#     x = traindesc[total].shape[0]
#     if (x != 100):
#         print(total)

[[ -6.66312291e-04   6.71038870e-03   4.40389570e-03 ...,   1.71688385e-03
    2.70135794e-03   4.50927718e-03]
 [  5.06966724e-04  -5.20876783e-04   1.97955617e-03 ...,  -1.60265504e-03
    5.52078988e-03   2.23441399e-03]
 [  5.57354651e-04  -6.60398975e-04   2.47966475e-03 ...,   3.27602378e-04
    5.70032001e-03   1.62721984e-03]
 ..., 
 [  7.18817872e-04   4.17748204e-04   2.15139496e-03 ...,  -6.47700764e-03
    1.99560286e-03   6.65072631e-03]
 [ -1.05403434e-03   5.09498594e-03   1.23607880e-03 ...,  -5.38383727e-04
    6.92025339e-03   1.43186247e-03]
 [ -5.12605766e-03  -8.06321186e-05   7.72237917e-03 ...,   1.57825547e-04
    4.50277980e-03   2.84625043e-04]]
0.000951505
(100, 64)
finish


In [142]:
# test SVM
# X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
# y = np.array([1, 1, 2, 2])
# clf = svm.SVC()
# clf.fit(X, y) 

# print(clf.predict([[-0.8, -1]]))

# test HoG
# height = trainset2[14].shape[0]
# width = trainset2[14].shape[1]

# location = (int(height/2), int(width/2), 3)
# print(location)

hogging = cv2.HOGDescriptor((64,64), (16,16), (8,8), (8,8), 9)
# im = cv2.imread(trainset2[2])
for do in range (5,15):
    height = trainset2[do].shape[0]
    width = trainset2[do].shape[1]
    location = (int(height/2), int(width/2))
    print(location)
    h = hogging.compute(trainset2[do], (8,8), (8,8), (location,))
    
    print(h)
    print(h.shape)

# test skhog
# skhog = hog(trainset2[2])
# print (skhog)

(166, 250)
[[ 0.08899941]
 [ 0.13387509]
 [ 0.0820311 ]
 ..., 
 [ 0.29399613]
 [ 0.22818741]
 [ 0.13744286]]
(1764, 1)
(167, 250)
[[ 0.08195453]
 [ 0.12414745]
 [ 0.18686917]
 ..., 
 [ 0.05053973]
 [ 0.059157  ]
 [ 0.07796087]]
(1764, 1)
(166, 250)
[[ 0.00752771]
 [ 0.0220084 ]
 [ 0.0473929 ]
 ..., 
 [ 0.00367893]
 [ 0.00184664]
 [ 0.00114828]]
(1764, 1)
(167, 250)
[[ 0.08263247]
 [ 0.07906519]
 [ 0.13760264]
 ..., 
 [ 0.22353849]
 [ 0.13511227]
 [ 0.14259522]]
(1764, 1)
(190, 250)
[[ 0.04907309]
 [ 0.06293432]
 [ 0.18095022]
 ..., 
 [ 0.23312981]
 [ 0.2050572 ]
 [ 0.23312981]]
(1764, 1)
(250, 204)
[[ 0.12916747]
 [ 0.22421856]
 [ 0.22421856]
 ..., 
 [ 0.19708879]
 [ 0.26470926]
 [ 0.26470926]]
(1764, 1)
(150, 128)
[[ 0.03344557]
 [ 0.09647261]
 [ 0.09159887]
 ..., 
 [ 0.10876709]
 [ 0.23028636]
 [ 0.22092351]]
(1764, 1)
(250, 166)
[[ 0.03212163]
 [ 0.14861102]
 [ 0.2628791 ]
 ..., 
 [ 0.17195301]
 [ 0.18554565]
 [ 0.13262519]]
(1764, 1)
(198, 250)
[[ 0.09670918]
 [ 0.25183177]
 [ 0.25